In [1]:
#import lib
import pandas as pd
import numpy as np

In [2]:
#read dataset
data = pd.read_csv('contracts_3_31_2017_partial_clean.csv' ,encoding =  "ISO-8859-1")
#data = pd.read_csv('contracts_4_4_2017_partial_clean.csv')

In [3]:
data.head()

,procuring_entity,description,amount,procuring_method,advert_date,tenders_sold,bids_received,type_of_procuring_entity,ad_date_months
0,kenya industrial research development institute,provision of inpatient medical insurance cover,19538935.0,open tender,2014-06-26,16.0,16.0,parastatal,6
1,kenya industrial research development institute,provision of security services,7864800.0,open tender,2014-06-26,17.0,17.0,parastatal,6
2,kenya industrial research development institute,audio visual installation works for the propos...,179900280.0,open tender,2014-06-26,7.0,5.0,parastatal,6
3,kenya industrial research development institute,lift and escalator installation works for the ...,119532786.0,open tender,2014-06-26,6.0,6.0,parastatal,6
4,kenya industrial research development institute,"supply, installation, training and commissioni...",220260000.0,open tender,2015-01-02,-19.0,-19.0,parastatal,1


In [4]:
#Check invalid data bids_received>tenders_sold <- invalid
len(data[data['bids_received'] > data['tenders_sold']])

488

In [5]:
#Replace bids_received > tenders_sold 
data.ix[data.tenders_sold < data.bids_received ,['tenders_sold','bids_received']] = np.nan;
data.ix[:5]

,procuring_entity,description,amount,procuring_method,advert_date,tenders_sold,bids_received,type_of_procuring_entity,ad_date_months
0,kenya industrial research development institute,provision of inpatient medical insurance cover,19538935.0,open tender,2014-06-26,16.0,16.0,parastatal,6
1,kenya industrial research development institute,provision of security services,7864800.0,open tender,2014-06-26,17.0,17.0,parastatal,6
2,kenya industrial research development institute,audio visual installation works for the propos...,179900280.0,open tender,2014-06-26,7.0,5.0,parastatal,6
3,kenya industrial research development institute,lift and escalator installation works for the ...,119532786.0,open tender,2014-06-26,6.0,6.0,parastatal,6
4,kenya industrial research development institute,"supply, installation, training and commissioni...",220260000.0,open tender,2015-01-02,-19.0,-19.0,parastatal,1
5,kenya industrial research development institute,"supply, installation , training and commission...",129757720.0,open tender,2015-01-02,NaN,NaN,parastatal,1


In [6]:
#Check invalid data bids_received or tenders_sold < 0
data[data['bids_received'] < 0]

,procuring_entity,description,amount,procuring_method,advert_date,tenders_sold,bids_received,type_of_procuring_entity,ad_date_months
4,kenya industrial research development institute,"supply, installation, training and commissioni...",220260000.0,open tender,2015-01-02,-19.0,-19.0,parastatal,1


In [7]:
#Replace tenders_sold <0 
data.ix[data.tenders_sold <0 ,['tenders_sold','bids_received']] = np.nan;

In [ ]:
print(len(data[data['bids_received']==0]),len(data[data['tenders_sold']==0]))


In [ ]:
data['tenders_sold'] = data['tenders_sold'].replace({ 0 : np.nan })
data['bids_received'] = data['bids_received'].replace({ 0 : np.nan })


In [ ]:
data.groupby(["tenders_sold","bids_received"]).size()

In [ ]:
#Not to use IQR Method to find IQR
q1 = data['tenders_sold'].quantile(.25)
q3 = data['tenders_sold'].quantile(.75)
iqr = q3-q1
print(iqr)

In [ ]:
def return_outliers_via_IQR(series):
    q1 = series.dropna().quantile(.25)
    q3 = series.dropna().quantile(.75)
    iqr = q3 - q1
    
    return (series < q1 - (1.5 * iqr)) | (series > q3 + (1.5 * iqr))

In [ ]:
df_mask = return_outliers_via_IQR(data.tenders_sold)
data[df_mask]

In [ ]:
#Find outliers after remove invalid data
data[data['tenders_sold'] > data['tenders_sold'].std() * 3]

## No need to replace outliers; only target variable would be problematic

In [ ]:
#Replace tenders_sold outliers
# data.ix[data['tenders_sold'] > data['tenders_sold'].std() * 3 ,['tenders_sold','bids_received']] = np.nan;

In [ ]:
#Didnt remove it as i think those records are not outliers
data[data['bids_received'] > data['bids_received'].std() * 3]

In [ ]:
data.isnull().sum()

In [ ]:
print('procuring_method_vars=>', data['procuring_method'].unique(),'\n')
print('type_of_procuring_entity_vars=>', data['type_of_procuring_entity'].unique())

In [ ]:
# Transform categorical variable to numeric variable
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['procuring_method_enc'] = lb.fit_transform(data.procuring_method)
data['type_of_procuring_entity_enc'] = lb.fit_transform(data.type_of_procuring_entity)

In [ ]:
data.head()

In [ ]:
#Split data into 70,30
# Cathy

np.random.seed(123) # get constant vals across the board

is_test = np.random.uniform(0, 1, len(data)) > 0.70
train = data[is_test==False]
test = data[is_test==True]




In [ ]:
len(train), len(test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

tender_imputer = KNeighborsRegressor(n_neighbors=5)
bids_imputer = KNeighborsRegressor(n_neighbors=5)

In [ ]:
#Split the training data set 
train_w = train[train.tenders_sold.isnull()==False]
train_w_null = train[train.tenders_sold.isnull()==True]


In [ ]:
#Create features
cols= ['amount', 'ad_date_months','procuring_method_enc','type_of_procuring_entity_enc']

In [ ]:
tender_imputer.fit(train_w[cols], train_w.tenders_sold)

In [ ]:
tenders_sold_new = tender_imputer.predict(train_w_null[cols])
tenders_sold_new[:5]

In [ ]:
bids_imputer.fit(train_w[cols], train_w.bids_received)

In [ ]:
bids_received_new = bids_imputer.predict(train_w_null[cols])
bids_received_new[:5]

In [ ]:
#Replace the missing value in training data
#Round to nearest integer
train_w_null['tenders_sold'] = np.rint(tenders_sold_new)
train_w_null['bids_received'] = np.rint(bids_received_new)


In [ ]:
train_w_null.isnull().sum()

In [ ]:
#Fill in missing values in testing data 
test_w_null = test[test.tenders_sold.isnull()==True]
test_w = test[test.tenders_sold.isnull()==False]
len(test_w_null) , len(test_w)

In [ ]:
tenders_sold_test = tender_imputer.predict(test_w_null[cols])
bids_received_test = bids_imputer.predict(test_w_null[cols])

In [ ]:
test_w_null['tenders_sold'] = np.rint(tenders_sold_test)
test_w_null['bids_received'] = np.rint(bids_received_test)

In [ ]:
test_w_null.isnull().sum()

In [ ]:
#combine the data back together
#Train dataset
train = train_w.append(train_w_null)
#Test dataset
test = test_w.append(test_w_null)
len(train),len(test)

In [ ]:
data2= train.append(test)
len(data2)

In [ ]:
data2.isnull().sum()

In [ ]:
# data2 = data2.drop("description",axis=1)

In [ ]:
data2.to_csv("data_cleaning_cathy.csv",index=False,encoding =  "utf-8")

## Text feature extraction

In [8]:
import pandas as pd
import numpy as np

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

In [10]:
contracts = pd.read_csv("data_cleaning_cathy.csv")

In [11]:
contracts.head()

,procuring_entity,description,amount,procuring_method,advert_date,tenders_sold,bids_received,type_of_procuring_entity,ad_date_months,procuring_method_enc,type_of_procuring_entity_enc
0,kenya industrial research development institute,provision of inpatient medical insurance cover,19538935.0,open tender,2014-06-26,16.0,16.0,parastatal,6,0,7
1,kenya industrial research development institute,provision of security services,7864800.0,open tender,2014-06-26,17.0,17.0,parastatal,6,0,7
2,kenya industrial research development institute,audio visual installation works for the propos...,179900280.0,open tender,2014-06-26,7.0,5.0,parastatal,6,0,7
3,kenya industrial research development institute,lift and escalator installation works for the ...,119532786.0,open tender,2014-06-26,6.0,6.0,parastatal,6,0,7
4,numerical machining complex,"supply, delivery, installation, testing, train...",76500000.0,open tender,2015-01-30,19.0,19.0,parastatal,1,0,7


In [12]:
contracts.isnull().sum()

procuring_entity                0
description                     0
amount                          0
procuring_method                0
advert_date                     0
tenders_sold                    0
bids_received                   0
type_of_procuring_entity        0
ad_date_months                  0
procuring_method_enc            0
type_of_procuring_entity_enc    0
dtype: int64

In [13]:
# Length (in chars) of the contract with the largest description
max([len(desc) for desc in contracts.description])

539

In [14]:
min([len(desc) for desc in contracts.description])

15

In [15]:
[(i,len(desc))for i, desc in enumerate(contracts.description) if len(desc) == 532]

[]

In [16]:
contracts.description[:5]

0       provision of inpatient medical insurance cover
1                       provision of security services
2    audio visual installation works for the propos...
3    lift and escalator installation works for the ...
4    supply, delivery, installation, testing, train...
Name: description, dtype: object

In [17]:
contracts.description=contracts.description.apply(lambda x: x.replace("\r","").replace("\n","").replace("\t",""))

## The things that we will miss such as below; which isn't exactly a 'description' 

In [18]:
contracts.description[1796]

'a restricted tender was floated to five prequalified suppliers for an advertising agent and only two responded as below:vendor       bid sum (vat inclusive) jwt                kshs 13,114,554transcendkshs 6,315,910considering the two bidders who submitted their bids were pre-qualified, the main criteria used in evaluation was cost.based on the above, the evaluation committee recommended award of the tender to transcend media group on contract basis considering they had submitted the overall lowest cost.'

In [19]:
tfv = TfidfVectorizer(min_df=3, strip_accents='unicode', analyzer='word',\
token_pattern=r'\w{1,}', ngram_range=(1,2), use_idf = 1, smooth_idf = 1, sublinear_tf = 1, stop_words='english')

In [20]:
description_data = tfv.fit_transform(list(contracts.description))

In [21]:
type(description_data)

scipy.sparse.csr.csr_matrix

In [22]:
description_data.shape

(2407, 2360)

In [23]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 120,random_state=123)
svd.fit(description_data)

TruncatedSVD(algorithm='randomized', n_components=120, n_iter=5,
       random_state=123, tol=0.0)

In [24]:
description_data = svd.transform(description_data)

In [25]:
description_data.shape

(2407, 120)

In [26]:
contracts.columns

Index(['procuring_entity', 'description', 'amount', 'procuring_method',
       'advert_date', 'tenders_sold', 'bids_received',
       'type_of_procuring_entity', 'ad_date_months', 'procuring_method_enc',
       'type_of_procuring_entity_enc'],
      dtype='object')

## Data Prediction

In [27]:
from sklearn.preprocessing import OneHotEncoder

In [28]:
contracts_type_pe = pd.get_dummies(data=contracts.type_of_procuring_entity_enc, prefix="proc_ent")
contracts_type_pe = contracts_type_pe.drop('proc_ent_0', axis =1)

In [29]:
cols_to_drop = ['type_of_procuring_entity_enc','description','type_of_procuring_entity','advert_date','procuring_method','procuring_entity']
contracts  = pd.concat([contracts.drop(cols_to_drop,axis=1), contracts_type_pe], axis =1)

In [30]:
contracts.columns

Index(['amount', 'tenders_sold', 'bids_received', 'ad_date_months',
       'procuring_method_enc', 'proc_ent_1', 'proc_ent_2', 'proc_ent_3',
       'proc_ent_4', 'proc_ent_5', 'proc_ent_6', 'proc_ent_7', 'proc_ent_8',
       'proc_ent_9'],
      dtype='object')

In [31]:
combined_features = np.hstack((contracts.drop('amount', axis=1).as_matrix(),description_data))
combined_features.shape



(2407, 133)

In [32]:
# X = contracts[['tenders_sold', 'bids_received', 'ad_date_months',\
# 'procuring_method_enc', 'proc_ent_1', 'proc_ent_2', 'proc_ent_3',\
# 'proc_ent_4', 'proc_ent_5', 'proc_ent_6', 'proc_ent_7', 'proc_ent_8',\
# 'proc_ent_9']]

X = combined_features
X.shape

(2407, 133)

In [33]:
Y = contracts.amount
Y = Y.as_matrix()

In [34]:
from sklearn.cross_validation import KFold

/Users/sincheng/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [35]:
evaluation_size = 0.1
kf = KFold(len(contracts), round(1./evaluation_size), random_state=123)

In [36]:
kf

sklearn.cross_validation.KFold(n=2407, n_folds=10, shuffle=False, random_state=123)

In [37]:
# getting the split
train_indices, validation_indices = next(iter(kf)) # K => 1st run split 10 times

X_train, Y_train = X[list(train_indices)], Y[list(train_indices)]

X_validation, Y_validation = X[validation_indices], Y[validation_indices]

In [38]:
len(X_train), len(X_validation)

(2166, 241)

## Model Building

In [39]:
from sklearn.ensemble import RandomForestRegressor

In [40]:
from sklearn.model_selection import GridSearchCV

In [41]:
# Grid Search

param_grid = [
    {'bootstrap': [False], 'n_estimators': [50,100], 'max_features':['sqrt',0.2]},
  ]

rf_regressor= RandomForestRegressor(criterion='mae',n_jobs=-1,warm_start=True,min_samples_leaf=50,random_state=123)


grid_search = GridSearchCV(rf_regressor, param_grid, cv=5,
                           scoring='neg_mean_absolute_error')



In [42]:
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=50,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=123,
           verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'bootstrap': [False], 'n_estimators': [50, 100], 'max_features': ['sqrt', 0.2]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [43]:
cvres1 = grid_search.cv_results_
for mean_score, params in zip(cvres1["mean_test_score"], cvres1["params"]):
    print(-mean_score, params)

78154845.8914 {'bootstrap': False, 'max_features': 'sqrt', 'n_estimators': 50}
78026614.9279 {'bootstrap': False, 'max_features': 'sqrt', 'n_estimators': 100}
77960651.653 {'bootstrap': False, 'max_features': 0.2, 'n_estimators': 50}
77854087.1743 {'bootstrap': False, 'max_features': 0.2, 'n_estimators': 100}


In [36]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

78168912.634 {'bootstrap': False, 'max_features': 'sqrt', 'n_estimators': 50}
78016630.1524 {'bootstrap': False, 'max_features': 'sqrt', 'n_estimators': 100}
78035213.3574 {'bootstrap': False, 'max_features': 'sqrt', 'n_estimators': 200}
77914189.1084 {'bootstrap': False, 'max_features': 0.2, 'n_estimators': 50}
77989062.5705 {'bootstrap': False, 'max_features': 0.2, 'n_estimators': 100}
78017601.3537 {'bootstrap': False, 'max_features': 0.2, 'n_estimators': 200}
77997339.6631 {'bootstrap': False, 'max_features': 0.3, 'n_estimators': 50}
78166684.4572 {'bootstrap': False, 'max_features': 0.3, 'n_estimators': 100}
78061242.5723 {'bootstrap': False, 'max_features': 0.3, 'n_estimators': 200}


In [ ]:
from sklearn.metrics import median_absolute_error

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_predict = grid_search.predict(X_validation)

In [ ]:
len(y_predict)

In [ ]:
ms = mean_squared_error(Y_validation, y_predict)

In [ ]:
rmse = np.sqrt(ms)
rmse

In [ ]:
median_absolute_error(Y_validation, y_predict)

In [ ]:
np.mean(contracts.amount)